In [4]:
%matplotlib ipympl
from models import *
# from models import find_index_of_nearest
from impedance.preprocessing import readCSV, readBioLogic
from impedance.visualization import plot_nyquist,plot_bode
from impedance.models.circuits import CustomCircuit
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from arbitau import give_z_for_arbitary_tau, tau_plotter
from scipy.integrate import solve_bvp

In [9]:
@element(num_params=5,units=["","","","",""],overwrite=True)
def arbitaualha(p,f):
    # modify the number of parameters according to the torutosity function
    Rref, Q, alpha, a, b = p[0], p[1], p[2], p[3], p[4]
    # provide the definition for the arbitrary tortuosity function
    
    def tau(x):
        # return 5+np.tanh(x*10-3)+2*np.tanh(x*10-5)
        return 5+np.tanh(x*10-a)+2*np.tanh(x*10-b)
    
    def fun(x,y,lmdcos,lmdsin):
        return np.vstack((y[2]*tau(x),y[3]*tau(x),lmdcos*y[0]-lmdsin*y[1],lmdcos*y[1]+lmdsin*y[0]))

    def bc(ya,yb):
        return np.array([ya[0]-1,yb[2],yb[3],ya[1]])

    # Q=6.61e-4
    # Rref=194.56

    x=np.linspace(0,1,1000)
    y=np.zeros((4,x.size))
    # print(y)

    Z_list=np.zeros((np.size(np.array(f))),dtype=complex)
    # print(Z_list)
    ii=0
    for freq in np.array(f):
        w=2*np.pi*freq
        lmdcos=w**alpha*Q*Rref*np.cos(alpha*np.pi/2)
        lmdsin=w**alpha*Q*Rref*np.sin(alpha*np.pi/2)
        sol1 = solve_bvp(lambda x,y: fun(x,y,lmdcos=lmdcos,lmdsin=lmdsin), bc, x, y)
        y3=sol1.sol(x)[2]
        y4=sol1.sol(x)[3]
        Z=2/(-(1/(Rref))*(y3[0]+y4[0]*1j))
        Z_list[ii]=Z
        ii=ii+1

    return Z_list

In [10]:
#initial guesses for the TLM
#Double layer Capacitance inside porous electrode
Q=6.61e-4
alpha=1

R1=400
delta1=0.3
R2=400

R3=400
delta2=0.3


#initial guesses for the HFR intercept and for any semicircle like features in the EIS data
Rhfr=200
Rrxn=200
Qdl=6.61e-4
alpha2=1

#other (for arbitrary tortuosity profile)
Rref=200
a=4
b=5

#For Two-step provide R1,R2,Q,delta1,alpha
#For Three-step provide R1,R2,R3,Q,delta1,delta2,alpha
#For Linear provide R1,R2,Q,alpha
#For Inv-Linear provide R1,R2,Q,alpha

###For arbitary profile provide <what?>###
#write the profile that you want to fit
#tau should remain >=1 in the entire domain of the electrode
#for example, tau=1+np.sin(2*x)

In [11]:
cmodel1=CustomCircuit(initial_guess=[Rref,Q,1,a,b,Rhfr,Rrxn,Qdl,alpha2],circuit="arbitaualha_1-R_1-p(R_2,CPE_1)")

In [12]:
freq_list=np.logspace(-1,2,100)
zpred=cmodel1.predict(freq_list)

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\impedance\models\circuits\circuits.py:144: UserWarning: Simulating circuit based on initial parameters
  warnings.warn("Simulating circuit based on initial parameters")


In [ ]:
import csv

# open the file in the write mode
with open('./zdata3.csv', 'w') as f:
    # create the csv writer
    for fq1,z in zip(freq_list,zpred):
        row=[fq1,float(np.real(z)),float(np.imag(z))]
        # write a row to the csv file
        writer = csv.writer(f)
        writer.writerow(row)